### How the model works
Documentation is provided for each function to make the model more understandable.Refer to it.
for more information refer to pages ( - ) in survey.pdf( ) -Will be updated soon

This notebook is an compilation of every function created to implement the model.

In [ ]:
def preImport(k2tpath):
        global clasr,happy_tt,args
        import pickle
        f=open('finalized_model.sav', 'rb')
        clasr = pickle.load(f)
        f.close()
        from gramformer import Gramformer
        global gf 
        gf=Gramformer(models = 1, use_gpu=False)
        import torch
        def set_seed(seed):
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(seed)
        set_seed(1212)
        global model
        from keytotext import trainer
        model=trainer()
        model.load_model(k2tpath, use_gpu=False)
        

----------------------------

### 1)Cleaning and dataframe generation

**clean(text,removestop)**

returns a cleaned text of the article/summary, expanding contractions such as ain't->am not, removing details of publishing date, published websites,stopwords(if passed text is an article)..etc

In [ ]:
def clean(text,removestop=True):
    from nltk.corpus import stopwords
    import re
    import contractions
    stops = stopwords.words('english')
    text = text.lower()
    text = text.split()
    tmp = []
    #expanding contractions
    for word in text:
        if((not removestop) or (word not in stops)):
            tmp.append(contractions.fix(word)) 
    text = ' '.join(tmp)
    
    #Substituting unncessary words using re.sub
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text=re.sub(' est ','',text)
    return text

**sentCreator(text)**

when passed a string text, returns sentences of the text as a single string. the produced string can be split into different sentences by using .split(".")

In [ ]:
def sentCreator(text):
    from nltk.tokenize import sent_tokenize
    sentences=""
    for sent in sent_tokenize(text):
        #less than 6 words in main article indicates very less important sentence, we can effectively ignore them
        if(len(sent.split())>5):
            sentences+=sent
    return sentences
    #Sentences generated from tokenize end with . , we can use this info to just create list of sentences by using .split(.)

**createCleaned(filename/path,training=True)**

when the path of the file/filename (.csv file) is passed, createCleaned will save a csv file titled "Cleaned.csv". 

**Note** Formats required and produced under training and testing:

**+** Under training: 

passed dataframe:  |--unused column--|--Articles--|--Summaries--|

produced dataframe: |--Index--|--Articles--|--Summaries--|--Sentences--|

**+** Under prediction/testing: 

passed dataframe:  |--unused column--|--Articles--|

produced dataframe: |--Index--|--Articles--|--Sentences--|

In [ ]:
def createCleaned(filename,training=True,noFile=False):
        import pandas as pd
        import numpy as np
        df=pd.DataFrame()
        if(noFile):
            df=filename
            article=df.iloc[:,1].values
        else:
            df=pd.read_csv(filename)
            article=df.iloc[:,1].values
        if(training):
            train_summary=df.iloc[:,2].values
        ta=[]
        ts=[]
        sen=[]
        fiv_percent=int(len(article)*0.05)+1
        for x in range(0,len(article)):
            ta.append(clean(article[x]))
            sen.append(sentCreator(article[x]).lower())
            if(training):
                ts.append(clean(train_summary[x],False))
            if(x%fiv_percent==0):
                #to track progress
                print(x," out of ",len(article)," parsed")
        cleaned=pd.DataFrame()
        cleaned["Articles"]=ta
        if(training):
            cleaned["Summaries"]=ts
        cleaned["Sentences"]=sen
        cleaned.drop_duplicates(inplace=True)
        cleaned.dropna(inplace=True)
        length=len(cleaned)
        index=[x for x in range(0,length)]
        cleaned["Index"]=index
        cleaned.set_index("Index",inplace=True)
        cleaned.to_csv("Cleaned.csv")
        return True

-----------------------------------

### 2)Extracting features and creating a supervised learning model, to get probabilities of a word being present in summary

**importCleaned(filename/path)**

returns dataframe under passed filepath, can be used to retrieve dataframe from createCleaned function

In [ ]:
def importCleaned(filename):
    import pandas as pd
    df=pd.read_csv(filename)
    return df

**absScore(text)**

given an input text, absScore returns a "word":AbsoluteScore dictionary. Absolutescore(x) denotes the Probability of the picked word being x, when you select a random word from the given text.

In [ ]:
def absScore(text):
    words_freq={}
    words=text.split()
    total=0
    for w in words:
        total+=1
        try:
            words_freq[w]+=1
        except:
            words_freq[w]=1  
    for i in words_freq.keys():
        words_freq[i]/=total
    return words_freq    

**sentScore(words,sentences)**

given the inputs: word-AbsoluteScore dictionary  &  string sentences to which these words belong to(sentences end with "." -ie format produced from sentCreator function),sentScore will return a dictionary of form sentence_number-sentenceScore.

sentenceScore (or) Score of a sentence is the average word probability(absScore) in that sentence

In [ ]:
def sentScore(words,sentences):
    sents=sentences.split(".")
    sentscore={}
    keys=words.keys()
    s=0
    for i in range(0,len(sents)):
        score=0
        itrsent=sents[i].split()
        itrsent_indict=[]
        for j in itrsent:
            if(j in keys):
                itrsent_indict.append(1)
            else:
                itrsent_indict.append(0)
        n=0
        for j in range(0,len(itrsent)):
            n+=1
            if(itrsent_indict[j]):
                score+=words[itrsent[j]]  
        if(n==0):
            sentscore[i]=0
        else:
            score/=n
            sentscore[i]=score
        s+=score
    for i in sentscore.keys():
        sentscore[i]/=s
    return sentscore

**inFirst(words,sentences)**

given the inputs: word-AbsoluteScore dictionary  &  string sentences to which these words belong to(each sentence ends with "." -ie format produced from sentCreator function) , inFirst will return a dictionary of form word-inFirstsentence, where inFirstsentence takes the value 0(word not in first sentence) or 1 (word in first sentence)


In [ ]:
def inFirst(words,sentences):
    inFirst={}
    sent=(sentences.split("."))[0]
    for i in words.keys():
        if(i in sent):
            inFirst[i]=1
        else:
            inFirst[i]=0
    return inFirst

**mostProminent(words,sentences,senrank)**

given the inputs: word-AbsoluteScore dictionary , string sentences to which these words belong to, and sentence-sentScore dictionary, mostProminent will return a word-mostProminentSentence dictionary . mostProminent sentence is the sentence in which the corresponding word occurs the most.

In [ ]:
def mostProminent(words,sentences,senrank):
    sent=sentences.split(".")
    mostProminent={}
    for i in words.keys():
        mps=0
        counts=0
        for j in range(0,len(sent)):
            c=sent[j].count(i)
            if(c>counts or ((c==counts)and senrank[j]>senrank[mps])):
                counts=c
                mps=j
        mostProminent[i]=mps
    return mostProminent  

**relScore(absscore,senscore,mostprominent)**

given the inputs word-AbsoluteScore dictionary , sentence-sentScore dictionary and word-mostProminent dictionary, relScore will return a word-relative_score dictionary. where relativeScore of word w is given by :absScore(word)* sentScore(most prominent sentence(word))

In [ ]:
def relScore(absscore,senscore,mostprominent):
    relscore={}
    for i in absscore.keys():
        relscore[i]=(absscore[i]*senscore[mostprominent[i]])
    return relscore

**inSummary(absscore,summary,training=True)**

inSummary takes in 2 inputs- word-Absoloute score dictionary, the corresponding summary to which the word dictionary belongs to. inSummary acts differently based on input given to the parameter training(True/False)

**+** training is True(default case):

will return a word-inSummary dictionary. inSummary produces values 0(if word is not in summary) or 1(if word is in summary)

**+** training is False:

will return a word-0 dictionary. 


In [ ]:
def inSummary(absscore,summary,training=True):
    inSummary={}
    if(training):
        for i in absscore.keys():
            if i in summary:
                inSummary[i]=1
            else:
                inSummary[i]=0
    else:
        for i in absscore.keys():
            inSummary[i]=0
    return inSummary

**summLen(absscore,summary=" ",default_len=200,training=True)**


summLen takes in 4 inputs- word-Absoloute score dictionary, the corresponding summary to which the word dictionary belongs to( by default-" ")and length( by default 200). summLen acts differently based on input given to the parameter training (either True/False).

**+** training is True(default case):

will return a word-summLen dictionary. summLen produces wordcount of the respective summary , the word belongs to.

**+** training is False:

will return a word-default_len dictionary. based on value passed to default_len.


In [ ]:
def summLen(absscore,summary=" ",default_len=200,training=True):
    summlen={}
    if(training):
        length=len(summary.split())
        for i in absscore.keys():
            summlen[i]=length
    else:
        for i in absscore.keys():
            summlen[i]=default_len
    return summlen

**dfCreator(df,start,stop,training=True,length=200)**

dfCreator acts differently based on values passes to training(T/F).

**+** training is True(default case):

will return a dataframe with the structure

*|  Index  |  Words  |  Absolute score  |  Relative score  |  inFirst sentence  |  inSummary  |  Summary Length  |  Summary  |*

Where column 2(abs score) to 7 are obtained by passing it to respective functions declared above with value of training being True

**+** training is False:

will return a dataframe with structure

*|  Index  |  Words  |  Absolute score  |  Relative score  |  inFirst sentence  |  inSummary  |  Summary Length  |*

Where column 2(abs score) to 7 are obtained by passing it to respective functions declared above with value of training being False


In [ ]:
def dfCreator(df,start,stop,training=True,length=200):
    import pandas as pd
    import numpy as np
    indofsum=3
    if(not training):
        indofsum=2
    absscore=[]
    for i in range(start,stop):
        absscore.append(absScore(df.iloc[i,1]))
    sentscore=[]
    for i in range(start,stop):
        sentscore.append(sentScore(absscore[i-start],df.iloc[i,indofsum]))
    infirst=[]
    for i in range(start,stop):
        infirst.append(inFirst(absscore[i-start],df.iloc[i,indofsum]))
    mp=[]
    for i in range(start,stop):
        mp.append(mostProminent(absscore[i-start],df.iloc[i,indofsum],sentscore[i-start]))
    relscore=[]
    for i in range(start,stop):
        relscore.append(relScore(absscore[i-start],sentscore[i-start],mp[i-start]))
    insum=[]
    if(training):
        for i in range(start,stop):
            insum.append(inSummary(absscore[i-start],df.iloc[i,2]))
    else:
        for i in range(start,stop):
            insum.append(inSummary(absscore[i-start],0,False))
    summlen=[]
    if(training):
        for i in range(start,stop):
            summlen.append(summLen(absscore[i-start],df.iloc[i,2]))
    else:
        for i in range(start,stop):
            summlen.append(summLen(absscore[i-start],default_len=length,training=False))
        
    final=[]
    summaries=df.iloc[:,-2].values
    summforkey=[]
    index=[]
    for i in range(0,len(relscore)):
        keys=absscore[i].keys()
        #structure of final: [word,[feautures]]
        for j in keys:
            if(training):
                summforkey.append(summaries[start+i])
            index.append(i)
            currkey=j
            feature=[absscore[i][currkey],relscore[i][currkey],infirst[i][currkey],insum[i][currkey],summlen[i][currkey]]
            final.append([currkey,feature,])
    words=[]
    abscore=[]
    relscore=[]
    infirst=[]
    insum=[]
    summlen=[]
    for i in range(0,len(final)):
        words.append(final[i][0])
        abscore.append(final[i][1][0])
        relscore.append(final[i][1][1])
        infirst.append(final[i][1][2])
        insum.append(final[i][1][3])
        summlen.append(final[i][1][4])
    newdf=pd.DataFrame()
    newdf["Index"]=index
    newdf["Words"]=words
    newdf["Absolute score"]=abscore
    newdf["Relative score"]=relscore
    newdf["In First sentence"]=infirst
    newdf["In Summary"]=insum
    newdf["Summary length"]=summlen
    if(training):
        newdf["Summary"]=summforkey
    return newdf
    
        

**NOTE** it is advised to run the function produceMLModel on collab/kaggle notebooks as for bigger dataset chances of getting Memoryrunouterror is high

**produceMLModel(df,start,stop)**

Produces a ML model which transforms and trains on the cleaned dataset df(cleaned via createCleaned) and outputs a RandomForest model saved as *finalized_model.sav* under the current directory. it is able to classify/predict probabilities of a word being in the summary given the features engineered using above functions.

In [ ]:
def produceMLModel(df,start,stop):
    import pandas as pd
    import numpy as np
    traindf=dfCreator(df,start,stop)
    from sklearn.ensemble import RandomForestClassifier
    x=traindf.iloc[:,[2,3,4,6]].values
    y=traindf.iloc[:,5].values
    clasr=RandomForestClassifier(n_estimators = 100, criterion = 'entropy',class_weight={0:0.95,1:0.05})
    clasr.fit(x,y)
    import pickle
    filename = 'finalized_model.sav'
    f=open(filename, 'wb')
    pickle.dump(clasr, f)
    f.close()
    


### 3)Generating sentences from set of words

**generateSet(df,start,stop,training=True,length=200,save_name="Keyword-text.csv")**

generates the best set of words for a given summary length( if testing ) , fixed summary length( taken from predefined summary during training) . more than several articles can be passed at once. produces a dataframe which has the structure:

*|    keywords    |    text    |* (if training. text is the corresponding summary of the given keywords)

*|    keywords    |    text    |* (if testing. Nan values for text, as it is not available)

the dataframe is stored as "Keyword-text.csv" by default.

if save_name is passed, it is stored as the string passed

In [ ]:
def generateSet(df,start=0,stop=1,training=True,length=300,save_name="Keyword-text.csv",new=False):
    import pandas as pd
    import numpy as np
    if(new):
        global clasr
        import pickle
        f=open('finalized_model.sav', 'rb')
        clasr = pickle.load(f)
    df=dfCreator(df,start,stop,training,length)
    if(not training):
        feauts=df.iloc[:,[2,3,4,6]].values
        probas=clasr.predict_proba(feauts)[:,1]
        df["Probability"]=probas
        df.drop("In Summary",axis=1,inplace=True)
        df.sort_values(by = ['Index', 'Probability'],ascending=False)
    else:
        df.drop("Summary length",axis=1,inplace=True)
    df.drop("Absolute score",axis=1,inplace=True)
    df.drop("Relative score",axis=1,inplace=True)
    df.drop("In First sentence",axis=1,inplace=True)
    f=open('finalized_model.sav', 'rb')
    
    #training: Index words inSummary summary
    #test: Index words Summarylength Summary probability(in descending denoting high proba)
    L=df.iloc[:,:].values
    keywords=[]
    summaries=[]
    i=0
    j=0
    while( i< len(L)):
        indexkey=L[i][0]
        j=i
        s=""
        length=lencovered=0
        if(training):
            while(j<len(L) and L[j][0]==indexkey):
                if(L[j][2]==1):
                    s+=(L[j][1]+" ")
                j+=1
            i=j
            summaries.append(L[j-1][3])
            keywords.append(s)
        if(not training):
            length=int(L[i][2]*0.7)
            while((j<len(L)) and (L[j][0]==indexkey) and lencovered<length):
                lencovered+=1
                s+=(L[j][1]+" ")
                j+=1
            summaries.append(" ")
            keywords.append(s)
            i=j
    keyword_textDf=pd.DataFrame()
    keyword_textDf["keywords"]=keywords
    keyword_textDf["text"]=summaries
    keyword_textDf.to_csv(save_name)

    
        
    

**Note** It is heavily recommended to use trainK2T function ONLY on kaggle/collab notebook as high amount of gpu processing power is required.

### Important: after installing keytotext package, go to their directory , open trainer.py and search for progress_bar_refresh_rate (use Ctrl+f) . remove that single parameter alone. 

Further info for editing from notebook: https://stackoverflow.com/a/30250285/18432853

**trainK2T(traindf,testdf,model_="t5-small")**

trainK2T gets the train and test dataframes created from generateSet to train the KeyTotext model on the specified transformers model_. The model is saved under current directory under the model folder.

In [ ]:
def trainK2T(traindf,testdf,model_="t5-small"):
    import pandas as pd
    test_df = pd.read_csv(testdf)
    train_df=pd.read_csv(traindf)
    train_df.drop("Unnamed: 0",axis=1,inplace=True)
    test_df.drop("Unnamed: 0",axis=1,inplace=True)
    s=[]
    for i in range(0,23792):
        s.append(" ")
    test_df.drop("text",axis=1,inplace=True)
    test_df["text"]=s
    from keytotext import trainer
    model=trainer()
    model.from_pretrained(model_name=model_)
    model.train(train_df=train_df, test_df=test_df,batch_size=5, max_epochs=2,use_gpu=True)
    model.save_model()

**The following 2  functions are to properly split the words into sentences**

In [ ]:
def findword(text):
    sentendwords=[".",",","?","!",";"]
    endwords={".":[],",":[],"?":[],"!":[],";":[]}
    length=len(text)
    for i in range(0,length):
        if(text[i] in sentendwords):
            j=i-length
            while(j>=(0-length) and text[j]!=" "):
                j-=1
            endwords[text[i]].append(text[j+1:i].lower())
    return endwords

            
            

In [ ]:
def createStops_andSplit(result):
    global input_field
    endwords=findword(input_field.get(1.0, "end-1c"))
    dotwords=endwords["."]
    commawords=endwords[","]
    qwords=endwords["?"]
    semicolwords=endwords[";"]
    excwords=endwords["!"]
    sentenders=endwords.keys()
    networds=dotwords+commawords+qwords+semicolwords+excwords
    splitted=result.split()
    ends=[]
    splitwords=[]
    currlength=0
    length=0
    i=0
    while("" in networds):
        networds.remove("")
    while(" " in networds):
        networds.remove(" ")
    while("  " in networds):
        networds.remove("  ")
    for word in splitted:
        length+=(len(word)+1)
        if(word in networds):
            if (word in dotwords ):
                ends.append(".")
            elif (word in commawords):
                ends.append(",")
            elif (word in qwords ):
                ends.append("?")
            elif (word in semicolwords ):
                ends.append(";")
            else :
                ends.append("!")
            splitwords.append(result[currlength:length]+ends[i])
            i+=1
            currlength=length
    if(currlength!=length):
        splitwords.append(result[currlength:len(result)]+".")
    return splitwords
    

**predictK2T(testdf,modelpath)**

will return summary predictions  for the test data frame uploaded using the model built from trainK2T and stored under modelpath. By default it is to be located under /model relative from current working directory, if doesnt, try to locate and give it as parameter.

In [ ]:
def predictK2T(testdf,modelpath=" ",gpu=True,onetime=True,new=False):
    global model,input_field
    if(new):
        from keytotext import trainer
        model=trainer()
        model.load_model(modelpath, use_gpu=gpu)
    kwords=testdf.iloc[:,1].values 
    predictions=[]
    correct_sents=""
    for i in range(0,len(kwords)):
        keywords=kwords[i]
        result=model.predict(keywords.split(),use_gpu=gpu)
        if(onetime):
            newresult=createStops_andSplit(result)
            for sents in newresult:
                correct_sents+=(" ".join(gf.correct(sents,max_candidates=1)))  
        else:
            correct_sents=gf.correct(result,max_candidates=1)
        predictions.append(correct_sents)
        if(onetime):
            return predictions[0]
    pred_df=pd.DataFrame()
    pred_df["Predictions"]=predictions
    pred_df.to_csv("PredictionsResult.csv")


In [ ]:
def finalfunc():
    global text_area
    global input_field
    global loading_label,gif
    import pandas as pd
    import tkinter as tk
    gif=create_loading_gif()
    loading_label.configure(image=gif)
    s=input_field.get(1.0, "end-1c")
    df=pd.DataFrame()
    df["unused"]=[0]
    df["Article"]=[s]
    createCleaned(df,training=False,noFile=True)
    df=pd.read_csv("Cleaned.csv")
    generateSet(df,training=False)
    df=pd.read_csv("Keyword-text.csv")
    s=predictK2T(df,gpu=False)
    text_area.delete("1.0", "end")
    text_area.insert("1.0",s)
    loading_label.configure(text="Generated!",width=20, height=20, bg="grey", font=("Arial", 12))

### 4)GUI

In [ ]:
def start_thread():
    # create and start a new thread
    thread = threading.Thread(target=finalfunc)
    thread.start()

In [ ]:
def create_loading_gif():
    global loading_gif
    import tkinter as tk
    # Create a PhotoImage object
    loading_gif = tk.PhotoImage(file="loading.gif")
    # Return the PhotoImage object
    return loading_gif

In [ ]:
def CreateInterface():
    import tkinter as tk
    # create the root window
    root = tk.Tk()
    root.title("Text Summarizer")
    root.configure(bg="grey")
    root.state('zoomed')
    # add three line breaks
    for i in range(3):
        tk.Label(root, text="", bg="grey").pack()
    # create the heading label
    heading = tk.Label(root, text="Abstractive Text Summarizer for News Articles" , bg="grey", font=("Arial", 20))
    heading.pack()
    # add three line breaks
    for i in range(3):
        tk.Label(root, text="", bg="grey").pack()
    # create a frame to hold the input field and text area
    frame = tk.Frame(root)
    frame.pack()
    # create the input field
    global input_field,text_area
    input_field = tk.Text(frame, width=60, height=20, font=("Arial", 12))
    input_field.pack(side="left")
    # create the text area
    text_area = tk.Label(frame, width=20, height=20, bg="grey", font=("Arial", 12))
    text_area.pack(side="left")
    # create the text area
    text_area = tk.Text(frame, width=60, height=20, font=("Arial", 12))
    text_area.pack(side="right")
    # add three line breaks
    for i in range(3):
        tk.Label(root, text="", bg="grey").pack()
    # create the submit button
    global submit_button
    submit_button = tk.Button(root, text="Generate", font=("Arial", 14),command=start_thread)
    submit_button.pack()
    # add one line break
    global loading_label
    for i in range(1):
        tk.Label(root, text="", bg="grey").pack()
    loading_label = tk.Label(root, text="",bg="grey")
    loading_label.pack()
    # add three line breaks
    for i in range(3):
        tk.Label(root, text="", bg="grey").pack()
    # create the subheading label
    subheading = tk.Label(root, text="Created with <3 by Jagannathan" , bg="grey", font=("Arial", 12))
    subheading.pack()
    # run the main loop
    root.mainloop()

### Example usage:

In [ ]:
import threading
import os
preImport(os.getcwd()+"/model")
#Path under which the t5 keytotext model is stored

In [ ]:
CreateInterface()